# Creating model without using Pipeline

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

import pickle

In [7]:
data = pd.read_csv('data/titanic/train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
data = data.drop(['PassengerId','Name','Ticket','Cabin'],axis = 1)
data.sample(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
613,0,3,male,NaN,0,0,7.7500,Q
238,0,2,male,19.0,0,0,10.5000,S
93,0,3,male,26.0,1,2,20.5750,S
875,1,3,female,15.0,0,0,7.2250,C
36,1,3,male,NaN,0,0,7.2292,C


In [9]:
data.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [12]:
X= data.drop('Survived',axis=1)
y=data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)

In [14]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
497,3,male,NaN,0,0,15.1000,S
98,2,female,34.0,0,1,23.0000,S
513,1,female,54.0,1,0,59.4000,C
882,3,female,22.0,0,0,10.5167,S
398,2,male,23.0,0,0,10.5000,S


* Sex , Embarked - nominal categorical - OneHotEncoder
* Age , Embarked - Missing Values - SimpleImputer

In [38]:
simp_age = SimpleImputer()
simp_emb = SimpleImputer(strategy='most_frequent')

X_train_age = simp.fit_transform(X_train[['Age']])
X_train_embarked = simp_emb.fit_transform(X_train[['Embarked']])
X_test_age = simp.fit_transform(X_test[['Age']])
X_test_embarked = simp_emb.fit_transform(X_test[['Embarked']])
X_train_age.shape , X_train_embarked.shape

((801, 1), (801, 1))

In [39]:
ohe_sex = OneHotEncoder(sparse=False,handle_unknown='ignore')
ohe_emb = OneHotEncoder(sparse=False,handle_unknown='ignore')
X_train_emb = ohe_emb.fit_transform(X_train_embarked)
X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_test_emb = ohe_emb.fit_transform(X_test_embarked)
X_test_sex = ohe_sex.fit_transform(X_test[['Sex']])
X_train_sex.shape , X_train_emb.shape
X_train_emb

array([[0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [28]:
X_train_left = X_train.drop(['Sex','Age','Embarked'],axis=1)
X_test_left = X_test.drop(['Sex','Age','Embarked'],axis=1)

In [40]:
X_train_new = np.concatenate((X_train_age,X_train_sex, X_train_emb,X_train_left),axis=1)
X_test_new = np.concatenate((X_test_age,X_test_sex, X_test_emb,X_test_left),axis=1)

In [41]:
X_train_new.shape

(801, 10)

In [48]:
clf = DecisionTreeClassifier()
clf.fit(X_train_new, y_train)
y_pred = clf.predict(X_test_new)
clf_acc = accuracy_score(y_test,y_pred)
print("Accuracy Score",clf_acc*100)

Accuracy Score 81.11111111111111


In [50]:
pickle.dump(ohe_sex, open('models/one_hot_sex.pkl','wb'))
pickle.dump(ohe_emb, open('models/one_hot_emb.pkl','wb'))
pickle.dump(clf, open('models/classification_titanic.pkl','wb'))

# How to predict from user data in this situation without using Pipeline ?

In [51]:
import pickle
import numpy as np

In [52]:
ohe_sex = pickle.load(open('models/one_hot_sex.pkl','rb'))
ohe_emb = pickle.load(open('models/one_hot_emb.pkl','rb'))
clf = pickle.load(open('models/classification_titanic.pkl','rb'))

In [55]:
test_input = np.array([2, 'male', 31.0, 0, 0, 10.5, 'S'], dtype=object).reshape(1,7)

In [56]:
test_input

array([[2, 'male', 31.0, 0, 0, 10.5, 'S']], dtype=object)

In [57]:
test_input_sex = ohe_sex.transform(test_input[:,1].reshape(1,1))
test_input_emb = ohe_emb.transform(test_input[:,-1].reshape(1,1))

C:\Users\91700\Desktop\sample_project_1\env\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [58]:
test_input_tranformed = np.concatenate((test_input[:,[0, 2, 3, 4, 5]], test_input_sex,test_input_emb), axis=1)

In [59]:
test_input_tranformed.shape

(1, 10)

In [60]:
clf.predict(test_input_tranformed)

array([1], dtype=int64)

# Model using Pipeline

In [214]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn import set_config
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [215]:
data = pd.read_csv('data/titanic/train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [216]:
data = data.drop(['PassengerId','Name','Ticket','Cabin'], axis=1)

In [217]:
X = data.drop("Survived", axis=1)
y = data["Survived"]

In [218]:
X_train,X_test, y_train, y_test= train_test_split(X,y,test_size=0.1)

In [219]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
820,1,female,52.0,1,1,93.50,S
260,3,male,NaN,0,0,7.75,Q
390,1,male,36.0,1,2,120.00,S
297,1,female,2.0,1,2,151.55,S
75,3,male,25.0,0,0,7.65,S


In [220]:
#imputer transformer
tfr1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_emb',SimpleImputer(strategy='most_frequent'),[6])],
      remainder= 'passthrough')

In [221]:
tfr2 = ColumnTransformer([
    ('one_hot_sex_emb', OneHotEncoder(sparse=False, handle_unknown='ignore'),[1,6])],
    remainder='passthrough')

In [222]:
tfr3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))])

In [223]:
tfr4 = SelectKBest(chi2, k=8)

In [224]:
trf5 = DecisionTreeClassifier()

In [225]:
pipe = make_pipeline(tfr1,tfr2,tfr3,tfr4,tfrf)

* we can use both pipeline or make_pipeline
* In Pipeline we need to give names while incase of make_pipeline we don't

In [226]:
pipe_model = Pipeline([ 
   ('tfr1',tfr1),
   ('tfr2',tfr2),
   ('tfr3',tfr3),
   ('tfr4',tfr4),
   ('trf5',tfrf)])
set_config(display='diagram')

In [227]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_emb',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one_hot_sex_emb',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x00000294753654C0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [228]:
y_pred = pipe.predict(X_test)
print("Accuracy Score ", accuracy_score(y_test,y_pred)*100)

Accuracy Score  58.88888888888889


In [229]:
cv_score = cross_val_score(pipe_model,X,y,cv=5, scoring='accuracy')
cv_score.mean()

0.6364258364195594

In [230]:
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [231]:
grid = GridSearchCV(pipe_model, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfr1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_emb',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('tfr2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('one_hot_sex_emb',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('tfr3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('tfr4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x00000294753654C0>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [232]:
grid.best_params_

{'trf5__max_depth': 1}

In [233]:
y_pred_cv = grid.predict(X_test)

In [234]:
print("Accuracy ",accuracy_score(y_test,y_pred_cv)*100)

Accuracy  58.88888888888889


In [237]:
pickle.dump(pipe,open('models/pipeline_titanic.pkl','wb'))

In [238]:
model = pickle.load(open('models/pipeline_titanic.pkl','rb'))

# How to predict from user data in this situation using Pipeline ?

In [239]:
test_input = np.array([2, 'male', 31.0, 0, 0, 10.5, 'S'], dtype=object).reshape(1,7)

In [240]:
y_pred = model.predict(test_input)
y_pred

C:\Users\91700\Desktop\sample_project_1\env\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
C:\Users\91700\Desktop\sample_project_1\env\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0], dtype=int64)